In [1]:
import pandas as pd
import numpy as np

In [2]:
address_clust = pd.read_csv('address_clust.csv', index_col='address_id')
address_clust.head()

,cluster_id
address_id,
71413451,1
71411914,1
71410369,1
71410100,1
71410040,1


In [3]:
address_clust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6996 entries, 71413451 to 56962221
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   cluster_id  6996 non-null   int64
dtypes: int64(1)
memory usage: 109.3 KB


In [4]:
address_stats = pd.read_csv('address_stats.csv', index_col='id')
address_stats.head()

,address_id,transaction_id,received,sent
id,,,,
87134765,46402336,19162323,0.0,1800000.0
87134766,45919235,19162323,0.0,1071052.0
87134767,46529090,19162323,1000000.0,0.0
87134768,46529091,19162323,1821052.0,0.0
87154706,46529090,19166856,0.0,1000000.0


In [5]:
address_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76724 entries, 87134765 to 139449245
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   address_id      76724 non-null  int64  
 1   transaction_id  76724 non-null  int64  
 2   received        76724 non-null  float64
 3   sent            76724 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 2.9 MB


Добавим колонку кластера в таблицу транзакций

In [6]:
address_stats['cluster_id'] = address_stats['address_id'].map(address_clust.to_dict()['cluster_id'])
address_stats['cluster_id'] = address_stats['cluster_id'].fillna(0).astype('int64')
address_stats.head()

,address_id,transaction_id,received,sent,cluster_id
id,,,,,
87134765,46402336,19162323,0.0,1800000.0,0
87134766,45919235,19162323,0.0,1071052.0,0
87134767,46529090,19162323,1000000.0,0.0,1
87134768,46529091,19162323,1821052.0,0.0,0
87154706,46529090,19166856,0.0,1000000.0,1


Построим перекрёстную таблицу:

In [7]:
df = address_stats.pivot_table(values=['received', 'sent'], index='transaction_id', columns='cluster_id', aggfunc='sum', fill_value=0)
df.head()

received                 sent            
cluster_id            0        1  2        0        1  2
transaction_id                                          
19162323        1821052  1000000  0  2871052        0  0
19166856         990000        0  0        0  1000000  0
19174421        4590000   200000  0  4800000        0  0
19190163         150000        0  0        0   200000  0
19205422        1671052  1000000  0  2721052        0  0

In [8]:
df['fee'] = df['sent'].apply(np.sum, axis=1) - df['received'].apply(np.sum, axis=1) 
df.head()

received                 sent                fee
cluster_id            0        1  2        0        1  2       
transaction_id                                                 
19162323        1821052  1000000  0  2871052        0  0  50000
19166856         990000        0  0        0  1000000  0  10000
19174421        4590000   200000  0  4800000        0  0  10000
19190163         150000        0  0        0   200000  0  50000
19205422        1671052  1000000  0  2721052        0  0  50000

Убедимся, что у всех транзакций только один отправитель:

In [9]:
all(df['sent'].apply(np.sum, axis=1) == df['sent'].apply(np.max, axis=1))

True

In [10]:
for i in range(3):
    temp_df = df[df[('sent',i)] > 0]
    print('кластер {} потратил {} биткоинов на fee'.format(i, temp_df['fee'].sum()/10**8))
    for j in range(3):
        if i != j:
            print('{} биткоинов отправил кластер {} в кластер {}'.format( temp_df[('received',j)].sum()/10**8, i, j))

кластер 0 потратил 0.95160384 биткоинов на fee
297.11968388 биткоинов отправил кластер 0 в кластер 1
9.74013861 биткоинов отправил кластер 0 в кластер 2
кластер 1 потратил 0.667144 биткоинов на fee
300.55392888 биткоинов отправил кластер 1 в кластер 0
129.24324 биткоинов отправил кластер 1 в кластер 2
кластер 2 потратил 0.74566722 биткоинов на fee
4.86984 биткоинов отправил кластер 2 в кластер 0
135.82 биткоинов отправил кластер 2 в кластер 1
